In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

pd.set_option("display.precision", 1)

# set a random seed
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


## On récupère la donnée

In [136]:
test_customers = pd.read_csv('data/test_customers.csv')
test_locations = pd.read_csv('data/test_locations.csv')

train_customers = pd.read_csv('data/train_customers.csv')
train_locations = pd.read_csv('data/train_locations.csv')

orders = pd.read_csv('data/orders.csv')
vendors = pd.read_csv('data/vendors.csv')


/tmp/ipykernel_7897/3811830009.py:7: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('data/orders.csv')


In [137]:
print('train_customers shape: ', train_customers.shape)
print('test_customers shape: ', test_customers.shape)
train_customers.head()

train_customers shape:  (34674, 8)
test_customers shape:  (9768, 8)


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


In [138]:
print('train_locations shape: ', train_locations.shape)
print('test_locations shape: ', test_locations.shape)
train_locations.head()

train_locations shape:  (59503, 5)
test_locations shape:  (16720, 5)


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.7,-78.8
1,02SFNJH,1,NaN,1.7,0.8
2,02SFNJH,2,NaN,-0.5,0.7
3,RU43CXC,0,Home,0.1,0.4
4,BDFBPRD,0,NaN,2.5,0.7


In [139]:
print('orders shape: ', orders.shape)
orders.head()

orders shape:  (135303, 26)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


In [140]:
print('vendors shape: ', vendors.shape)
vendors.head()

vendors shape:  (100, 59)


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


### On merge les customer et leur localisation

In [141]:
# merge train_customers and train_locations
train = pd.merge(train_customers, train_locations, left_on='akeed_customer_id', right_on='customer_id', how='right')
train.drop(['akeed_customer_id'], axis=1, inplace=True)
train['CID X LOC'] = train['customer_id'] + ' X ' + train['location_number'].astype(str)

print('train shape: ', train.shape)
train.head()

train shape:  (59692, 13)


,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude,CID X LOC
0,NaN,NaN,1.0,1.0,EN,2019-06-17 00:33:23,2019-10-01 18:46:33,02SFNJH,0,NaN,1.7,-78.8,02SFNJH X 0
1,NaN,NaN,1.0,1.0,EN,2019-06-17 00:33:23,2019-10-01 18:46:33,02SFNJH,1,NaN,1.7,0.8,02SFNJH X 1
2,NaN,NaN,1.0,1.0,EN,2019-06-17 00:33:23,2019-10-01 18:46:33,02SFNJH,2,NaN,-0.5,0.7,02SFNJH X 2
3,NaN,NaN,1.0,1.0,NaN,2020-01-21 21:19:21,2020-01-21 21:21:24,RU43CXC,0,Home,0.1,0.4,RU43CXC X 0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BDFBPRD,0,NaN,2.5,0.7,BDFBPRD X 0


In [142]:
# merge test_customers and test_locations
test = pd.merge(test_customers, test_locations, left_on='akeed_customer_id', right_on='customer_id')
test.drop(['akeed_customer_id'], axis=1, inplace=True)
test['CID X LOC'] = test['customer_id'] + ' X ' + test['location_number'].astype(str)

print('test shape: ', test.shape)
test.head()

test shape:  (16333, 13)


,gender,dob,status,verified,language,created_at,updated_at,customer_id,location_number,location_type,latitude,longitude,CID X LOC
0,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,0,NaN,-9.6e+01,-67.2,ICE2DJP X 0
1,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,1,NaN,3.9e-02,-78.6,ICE2DJP X 1
2,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,2,Other,-9.5e+01,43.7,ICE2DJP X 2
3,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,3,Other,-9.6e+01,43.6,ICE2DJP X 3
4,Male,NaN,1,1,EN,2018-02-07 16:45:36,2018-02-07 16:45:36,ICE2DJP,4,Work,2.0e+02,44.1,ICE2DJP X 4


### Depuis Order on veut recuperer de la data pour améliorer notre dataframe vendors

In [143]:
for col in orders.columns:
    print(col, orders[col].isnull().sum(), orders[col].dtype)

akeed_order_id 70 float64
customer_id 0 object
item_count 6925 float64
grand_total 0 float64
payment_mode 0 int64
promo_code 130998 object
vendor_discount_amount 0 float64
promo_code_discount_percentage 69423 float64
is_favorite 35195 object
is_rated 0 object
vendor_rating 90083 float64
driver_rating 0 float64
deliverydistance 0 float64
preparationtime 55560 float64
delivery_time 130180 object
order_accepted_time 48348 object
driver_accepted_time 88845 object
ready_for_pickup_time 51054 object
picked_up_time 51438 object
delivered_time 49562 object
delivery_date 99759 object
vendor_id 0 int64
created_at 0 object
LOCATION_NUMBER 0 int64
LOCATION_TYPE 48893 object
CID X LOC_NUM X VENDOR 0 object


In [144]:
# improve vendors df by adding data from orders df
# add number of orders per vendor
vendor_orders = orders.groupby('vendor_id')['CID X LOC_NUM X VENDOR'].count().reset_index()
vendor_orders.columns = ['vendor_id', 'vendor_orders']

# apply log function to vendor_orders
vendor_orders['vendor_orders'] = np.log(vendor_orders['vendor_orders'])

vendors = pd.merge(vendors, vendor_orders, right_on='vendor_id', left_on='id', how='left').drop(['vendor_id'], axis=1)

# add number of unique customers per vendor
vendor_customers = orders.groupby('vendor_id')['customer_id'].nunique().reset_index()
vendor_customers.columns = ['vendor_id', 'vendor_customers']

# apply log function to vendor_customers
vendor_customers['vendor_customers'] = np.log(vendor_customers['vendor_customers'])

vendors = pd.merge(vendors, vendor_customers, right_on='vendor_id', left_on='id', how='left').drop(['vendor_id'], axis=1)

for col in vendors.columns:
    print(col, vendors[col].isnull().sum(), vendors[col].dtype)
vendors.head()

id 0 int64
authentication_id 0 float64
latitude 0 float64
longitude 0 float64
vendor_category_en 0 object
vendor_category_id 0 float64
delivery_charge 0 float64
serving_distance 0 float64
is_open 0 float64
OpeningTime 9 object
OpeningTime2 9 object
prepration_time 0 int64
commission 15 float64
is_akeed_delivering 0 object
discount_percentage 0 float64
status 0 float64
verified 0 int64
rank 0 int64
language 15 object
vendor_rating 0 float64
sunday_from_time1 1 object
sunday_to_time1 1 object
sunday_from_time2 58 object
sunday_to_time2 58 object
monday_from_time1 0 object
monday_to_time1 0 object
monday_from_time2 58 object
monday_to_time2 58 object
tuesday_from_time1 1 object
tuesday_to_time1 1 object
tuesday_from_time2 59 object
tuesday_to_time2 59 object
wednesday_from_time1 0 object
wednesday_to_time1 0 object
wednesday_from_time2 58 object
wednesday_to_time2 58 object
thursday_from_time1 1 object
thursday_to_time1 1 object
thursday_from_time2 58 object
thursday_to_time2 58 object
fr

,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders,vendor_orders,vendor_customers
0,4,118597.0,-0.6,0.8,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1,7.7,6.8
1,13,118608.0,-0.5,0.7,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1,6.6,5.7
2,20,118616.0,-0.4,0.6,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,"Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1,7.4,6.6
3,23,118619.0,-0.6,0.8,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,"Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1,6.8,6.1
4,28,118624.0,0.5,0.6,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1,7.3,6.9


### Nettoyage de la donnée

Nettoyage des customer X location

In [145]:
# label encoding
from sklearn.preprocessing import LabelEncoder

# print train columns
print('train columns: ', train.columns)
print('train shape: ', train.shape)
print('number of missing values per column:')
display(train.isnull().sum())
print('df_input types: ', train.dtypes)

def clean_df(df, le):
    # no warning
    pd.options.mode.chained_assignment = None

    # df cleaning
    new_df = df[['CID X LOC', 'latitude', 'longitude', 'location_type', 'status', 'gender', 'language']]

    # we fill the missing values with the mean
    new_df['latitude'].fillna(new_df['latitude'].mean(), inplace=True)
    new_df['longitude'].fillna(new_df['longitude'].mean(), inplace=True)
    new_df['language'].fillna('EN', inplace=True)
    new_df['gender'].fillna('None', inplace=True)
    new_df['location_type'].fillna('None', inplace=True)

    # fit and transform the location_type column
    new_df['location_type'] = le.fit_transform(new_df['location_type'])
    new_df['language'] = le.fit_transform(new_df['language'])
    new_df['gender'] = le.fit_transform(new_df['gender'])

    # set back the warning
    pd.options.mode.chained_assignment = 'warn'

    return new_df


# create a LabelEncoder object
le = LabelEncoder()

train = clean_df(train, le)
test = clean_df(test, le)
train.head()

train columns:  Index(['gender', 'dob', 'status', 'verified', 'language', 'created_at',
       'updated_at', 'customer_id', 'location_number', 'location_type',
       'latitude', 'longitude', 'CID X LOC'],
      dtype='object')
train shape:  (59692, 13)
number of missing values per column:


gender             18530
dob                54497
status              1479
verified            1479
language           17248
created_at          1479
updated_at          1479
customer_id            0
location_number        0
location_type      27260
latitude               6
longitude              6
CID X LOC              0
dtype: int64

df_input types:  gender              object
dob                float64
status             float64
verified           float64
language            object
created_at          object
updated_at          object
customer_id         object
location_number      int64
location_type       object
latitude           float64
longitude          float64
CID X LOC           object
dtype: object


,CID X LOC,latitude,longitude,location_type,status,gender,language
0,02SFNJH X 0,1.7,-78.8,1,1.0,9,0
1,02SFNJH X 1,1.7,0.8,1,1.0,9,0
2,02SFNJH X 2,-0.5,0.7,1,1.0,9,0
3,RU43CXC X 0,0.1,0.4,0,1.0,9,0
4,BDFBPRD X 0,2.5,0.7,1,NaN,9,0


Nettoyage des vendors

In [146]:
# print vendors columns
print('vendors columns: ', vendors.columns)
print('vendors shape: ', vendors.shape)
print('number of missing values per column:')
display(vendors.isnull().sum())
print('df_input types: ', vendors.dtypes)

vendors = vendors[['id', 'latitude', 'longitude', 'vendor_category_id', 'delivery_charge', 'serving_distance', 
                   'is_open', 'prepration_time', 'discount_percentage', 'rank', 'language', 'vendor_rating', 'device_type', 'vendor_tag_name', 
                   'vendor_orders', 'vendor_customers'
                   ]]
# rename the id column to VENDOR
vendors.rename(columns={'id': 'VENDOR'}, inplace=True)


vendors['language'].fillna('EN', inplace=True)
vendors["vendor_tag_name"].fillna("", inplace=True)

vendors['language'] = le.fit_transform(vendors['language'])

vendors.head()

vendors columns:  Index(['id', 'authentication_id', 'latitude', 'longitude',
       'vendor_category_en', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'is_open', 'OpeningTime', 'OpeningTime2',
       'prepration_time', 'commission', 'is_akeed_delivering',
       'discount_percentage', 'status', 'verified', 'rank', 'language',
       'vendor_rating', 'sunday_from_time1', 'sunday_to_time1',
       'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1',
       'monday_to_time1', 'monday_from_time2', 'monday_to_time2',
       'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2',
       'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1',
       'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1',
       'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2',
       'friday_from_time1', 'friday_to_time1', 'friday_from_time2',
       'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1',
       'saturday_from

id                    0
authentication_id     0
latitude              0
longitude             0
vendor_category_en    0
                     ..
updated_at            0
device_type           0
display_orders        0
vendor_orders         0
vendor_customers      0
Length: 61, dtype: int64

df_input types:  id                      int64
authentication_id     float64
latitude              float64
longitude             float64
vendor_category_en     object
                       ...   
updated_at             object
device_type             int64
display_orders          int64
vendor_orders         float64
vendor_customers      float64
Length: 61, dtype: object


,VENDOR,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,language,vendor_rating,device_type,vendor_tag_name,vendor_orders,vendor_customers
0,4,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,0,4.4,3,"Arabic,Breakfast,Burgers,Desserts,Free Deliver...",7.7,6.8
1,13,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,0,4.7,3,"Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",6.6,5.7
2,20,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,0,4.5,3,"Breakfast,Desserts,Free Delivery,Indian",7.4,6.6
3,23,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,0,4.5,3,"Burgers,Desserts,Fries,Salads",6.8,6.1
4,28,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,0,4.4,3,Burgers,7.3,6.9


transform the last column into multiples column for each tag, filled with 0 or 1 if the tag is in the row

In [147]:
# split the vendor_tag_name strings into a list of strings
vendors['vendor_tag_name'] = vendors['vendor_tag_name'].str.split(',')

vendors.head()

,VENDOR,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,language,vendor_rating,device_type,vendor_tag_name,vendor_orders,vendor_customers
0,4,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,0,4.4,3,"[Arabic, Breakfast, Burgers, Desserts, Free De...",7.7,6.8
1,13,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,0,4.7,3,"[Breakfast, Cakes, Crepes, Italian, Pasta, Piz...",6.6,5.7
2,20,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,0,4.5,3,"[Breakfast, Desserts, Free Delivery, Indian]",7.4,6.6
3,23,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,0,4.5,3,"[Burgers, Desserts, Fries, Salads]",6.8,6.1
4,28,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,0,4.4,3,[Burgers],7.3,6.9


In [148]:
#create a vector with all the tags in vendor_tag_name with no duplicates
tags = []
for e in vendors['vendor_tag_name']:
    for tag in e:
        tags.append(tag.strip())

tags = list(set(tags))
tags.remove('')

print('number of tags: ', len(tags))
print(tags)

number of tags:  67
['Salads', 'Thali', 'Japanese', 'Biryani', 'Coffee', 'Indian', 'Pizza', 'Pastas', 'Hot Dogs', 'Rolls', 'Kushari', 'Pizzas', 'Dimsum', 'Frozen yoghurt', 'Seafood', 'Kebabs', 'Shuwa', 'Fatayers', 'Crepes', 'Pancakes', 'Pastry', 'Waffles', 'Combos', 'Family Meal', 'Organic', 'Hot Chocolate', 'Soups', 'Pasta', 'American', 'Mandazi', 'Manakeesh', 'Karak', 'Free Delivery', 'Sandwiches', 'Asian', 'Fresh Juices', 'Milkshakes', 'Fries', 'Burgers', 'Lebanese', 'Desserts', 'Healthy Food', 'Thai', 'Arabic', 'Churros', 'Grills', 'Smoothies', 'Cafe', 'Kids meal', 'Steaks', 'Mexican', 'Omani', 'Breakfast', 'Mojitos', 'Ice creams', 'Donuts', 'Mishkak', 'Spanish Latte', 'Sweets', 'Shawarma', 'Bagels', 'Sushi', 'Italian', 'Vegetarian', 'Rice', 'Chinese', 'Cakes']


In [149]:
# add a column for each tag in vendors
# fill the column with 1 if the tag is in vendor_tag_name
for tag in tags:
    vendors[tag] = vendors['vendor_tag_name'].apply(lambda x: 1 if tag in x else 0)

# drop the vendor_tag_name column
vendors.drop(['vendor_tag_name'], axis=1, inplace=True)

vendors.head()

,VENDOR,latitude,longitude,vendor_category_id,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,rank,...,Spanish Latte,Sweets,Shawarma,Bagels,Sushi,Italian,Vegetarian,Rice,Chinese,Cakes
0,4,-0.6,0.8,2.0,0.0,6.0,1.0,15,0.0,11,...,0,0,1,0,0,0,0,0,0,0
1,13,-0.5,0.7,2.0,0.7,5.0,1.0,14,0.0,11,...,0,0,0,0,0,1,0,0,0,1
2,20,-0.4,0.6,2.0,0.0,8.0,1.0,19,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,23,-0.6,0.8,2.0,0.0,5.0,1.0,16,0.0,11,...,0,0,0,0,0,0,0,0,0,0
4,28,0.5,0.6,2.0,0.7,15.0,1.0,10,0.0,11,...,0,0,0,0,0,0,0,0,0,0


### Fusion des deux vecteurs

ajout de la colonne prédiction pour pouvoir ensuite génerer y, ici on decide de set les valeurs dans prediction avec le rating donné par les utilisateurs, et par la moyenne de la note du vendeur sinon

In [150]:
# if is_rated is 0 then rating is nan
orders.loc[orders['is_rated'] == 0, 'vendor_rating'] = np.nan

# keep only the CID X LOC X VENDOR, the rating columns, and the column needed to merge
orders = orders[['CID X LOC_NUM X VENDOR', 'vendor_rating', 'customer_id', 'vendor_id', 'LOCATION_NUMBER']]

# rename the vendor_rating column to prediction and the CID X LOC_NUM X VENDOR column to CID X LOC X VENDOR
orders.rename(columns={'vendor_rating': 'prediction', 'CID X LOC_NUM X VENDOR': 'CID X LOC X VENDOR'}, inplace=True)

# create the CID X LOC column
orders['CID X LOC'] = orders['customer_id'] + ' X ' + orders['LOCATION_NUMBER'].astype(str)

# merge train and vendors
train = pd.merge(train, orders, on='CID X LOC').drop(['CID X LOC'], axis=1)

# merge train and vendors
train = pd.merge(train, vendors, left_on='vendor_id', right_on='VENDOR', how='left').drop(['vendor_id', 'VENDOR'], axis=1)

# if prediciton is NaN, it means that the customer didn't rate the vendor
# we fill the missing values with the same value as the vendor_rating column
train['prediction'].fillna(train['vendor_rating'], inplace=True)

# drop the NA values
train.dropna(inplace=True)

# drop the LOCATION_NUMBER column
train.drop(['LOCATION_NUMBER'], axis=1, inplace=True)

# drop the customer_id column
train.drop(['customer_id'], axis=1, inplace=True)

print('train shape: ', train.shape)
train.head()

train shape:  (132226, 89)


,latitude_x,longitude_x,location_type,status,gender,language_x,CID X LOC X VENDOR,prediction,latitude_y,longitude_y,...,Spanish Latte,Sweets,Shawarma,Bagels,Sushi,Italian,Vegetarian,Rice,Chinese,Cakes
0,1.7,0.8,1,1.0,9,0,02SFNJH X 1 X 199,0.0,2.3,0.7,...,0,0,0,0,0,0,0,0,0,1
1,1.7,0.8,1,1.0,9,0,02SFNJH X 1 X 855,4.2,2.1,0.7,...,0,0,0,0,0,0,0,0,0,0
2,1.7,0.8,1,1.0,9,0,02SFNJH X 1 X 192,4.3,1.7,0.8,...,0,0,1,0,0,0,0,0,0,0
3,-0.5,0.7,1,1.0,9,0,02SFNJH X 2 X 159,0.0,-0.6,0.8,...,0,0,0,0,0,0,0,0,0,0
4,0.1,0.4,0,1.0,9,0,RU43CXC X 0 X 856,4.3,0.3,0.5,...,0,0,0,0,0,0,0,0,0,0


In [151]:
# print the preparation time range
print('preparation time range: ', train['prepration_time'].min(), train['prepration_time'].max())

# print the delivery charge range
print('delivery charge range: ', train['delivery_charge'].min(), train['delivery_charge'].max())

preparation time range:  5 45
delivery charge range:  0.0 0.7


on ajoute de l'information dans notre colonne prédiction :
- on part du principe qu'un long temps d'attente baisse l'appréciation

In [152]:
# we add the preparation time to the prediction, a long preparation time result into a worse preparation time
# preparation time is between 0 and 45 so we can use this information
train['prediction'] = train['prediction'] + (45 - train['prepration_time']) / 45

# we add the delivery charge to the prediction, a high delivery charge result into a worse prediction
# delivery charge is between 0 and 0.7
train['prediction'] = train['prediction'] - train['delivery_charge']

In [153]:
from math import radians, cos, sin, asin, sqrt

# Haversine formula to calculate distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8 # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R*c

In [154]:
for col in train.columns:
    print(col, train[col].isnull().sum())

# clean the train dataframe

# add a column "same_language" then drop the language_x and language_y column
train['same_language'] = train['language_x'] == train['language_y']
train.drop(['language_x', 'language_y'], axis=1, inplace=True)

# add a column "is_in_delivery_zone" then drop the latitude_x, longitude_x, latitude_y, longitude_y and serving_distance column
train['is_in_delivery_zone'] = train.apply(lambda x: haversine(x['latitude_x'], x['longitude_x'], x['latitude_y'], x['longitude_y']) <= x['serving_distance'], axis=1)
train.drop(['latitude_x', 'longitude_x', 'latitude_y', 'longitude_y', 'serving_distance'], axis=1, inplace=True)

# convert same_language and is_in_delivery_zone to int
train['same_language'] = train['same_language'].astype(int)
train['is_in_delivery_zone'] = train['is_in_delivery_zone'].astype(int)


latitude_x 0
longitude_x 0
location_type 0
status 0
gender 0
language_x 0
CID X LOC X VENDOR 0
prediction 0
latitude_y 0
longitude_y 0
vendor_category_id 0
delivery_charge 0
serving_distance 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
language_y 0
vendor_rating 0
device_type 0
vendor_orders 0
vendor_customers 0
Salads 0
Thali 0
Japanese 0
Biryani 0
Coffee 0
Indian 0
Pizza 0
Pastas 0
Hot Dogs 0
Rolls 0
Kushari 0
Pizzas 0
Dimsum 0
Frozen yoghurt 0
Seafood 0
Kebabs 0
Shuwa 0
Fatayers 0
Crepes 0
Pancakes 0
Pastry 0
Waffles 0
Combos 0
Family Meal 0
Organic 0
Hot Chocolate 0
Soups 0
Pasta 0
American 0
Mandazi 0
Manakeesh 0
Karak 0
Free Delivery 0
Sandwiches 0
Asian 0
Fresh Juices 0
Milkshakes 0
Fries 0
Burgers 0
Lebanese 0
Desserts 0
Healthy Food 0
Thai 0
Arabic 0
Churros 0
Grills 0
Smoothies 0
Cafe 0
Kids meal 0
Steaks 0
Mexican 0
Omani 0
Breakfast 0
Mojitos 0
Ice creams 0
Donuts 0
Mishkak 0
Spanish Latte 0
Sweets 0
Shawarma 0
Bagels 0
Sushi 0
Italian 0
Vegetarian 0
Rice 0
Ch

In [155]:
for col in train.columns:
    print(col, train[col].isnull().sum())

# drop CID X LOC X VENDOR column
train.drop(['CID X LOC X VENDOR'], axis=1, inplace=True)

location_type 0
status 0
gender 0
CID X LOC X VENDOR 0
prediction 0
vendor_category_id 0
delivery_charge 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
vendor_rating 0
device_type 0
vendor_orders 0
vendor_customers 0
Salads 0
Thali 0
Japanese 0
Biryani 0
Coffee 0
Indian 0
Pizza 0
Pastas 0
Hot Dogs 0
Rolls 0
Kushari 0
Pizzas 0
Dimsum 0
Frozen yoghurt 0
Seafood 0
Kebabs 0
Shuwa 0
Fatayers 0
Crepes 0
Pancakes 0
Pastry 0
Waffles 0
Combos 0
Family Meal 0
Organic 0
Hot Chocolate 0
Soups 0
Pasta 0
American 0
Mandazi 0
Manakeesh 0
Karak 0
Free Delivery 0
Sandwiches 0
Asian 0
Fresh Juices 0
Milkshakes 0
Fries 0
Burgers 0
Lebanese 0
Desserts 0
Healthy Food 0
Thai 0
Arabic 0
Churros 0
Grills 0
Smoothies 0
Cafe 0
Kids meal 0
Steaks 0
Mexican 0
Omani 0
Breakfast 0
Mojitos 0
Ice creams 0
Donuts 0
Mishkak 0
Spanish Latte 0
Sweets 0
Shawarma 0
Bagels 0
Sushi 0
Italian 0
Vegetarian 0
Rice 0
Chinese 0
Cakes 0
same_language 0
is_in_delivery_zone 0


### Création des data splits

In [156]:
# creation of y_train
y_train = train['prediction']
train.drop(['prediction'], axis=1, inplace=True)
print('y_train shape: ', y_train.shape)

# print the number of each unique value in prediction column
print(y_train.value_counts())

y_train shape:  (132226,)
 4.2e+00    5933
 5.5e+00    5297
-3.3e-02    4894
 6.7e-01    4273
 4.3e+00    4070
            ... 
 8.8e-01       4
 3.3e+00       3
 1.8e+00       3
 1.9e+00       2
 2.3e+00       2
Name: prediction, Length: 180, dtype: int64


On veut un nombre égaux d'exemple dans les 2 classes

In [157]:
# if prediction is in the top 50% of the prediction, we set it to 1, else we set it to 0
median = y_train.median()
y_train = y_train.apply(lambda x: 1 if x > median else 0)

# Count the number of 0 and 1 in y
print(pd.Series(y_train).value_counts())

0    67541
1    64685
Name: prediction, dtype: int64


les valeurs ne sont pas équilibrée on va donc les scale pour ne pas biaiser l'ia

In [158]:
# Scale X_balanced
scalerX = StandardScaler()
x_train = scalerX.fit_transform(train)

In [159]:
"""from imblearn.under_sampling import RandomUnderSampler
# Use RandomUnderSampler to balance the classes
rus = RandomUnderSampler(random_state=42)
x_train, y_train = rus.fit_resample(train, y_train)
del train

# Print the class distribution of the balanced y
print(pd.Series(y_train).value_counts())"""

'from imblearn.under_sampling import RandomUnderSampler\n# Use RandomUnderSampler to balance the classes\nrus = RandomUnderSampler(random_state=42)\nx_train, y_train = rus.fit_resample(train, y_train)\ndel train\n\n# Print the class distribution of the balanced y\nprint(pd.Series(y_train).value_counts())'

In [160]:
# data split
X_train, X_test = train_test_split(x_train, test_size=0.2, random_state=42, shuffle=True)
print('test shape: ', X_test.shape)
print('train shape: ', X_train.shape)

Y_train, Y_test = train_test_split(y_train, test_size=0.2, random_state=42, shuffle=True)

test shape:  (26446, 82)
train shape:  (105780, 82)


### Creation du modèle

In [161]:
# creation of an ai model
# it takes as input a dataframe with the columns as in df_input
# it returns a dataframe with the columns as in all_pairs and a column PREDICTION with the prediction

from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

model = keras.Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               21248     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 62,465
Trainable params: 62,465
Non-trainable params: 0
_________________________________________________________________


2023-05-16 15:16:45.725686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-16 15:16:45.768990: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [162]:
# lr reducer callback
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, factor=0.2, min_lr=1e-7)

# early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(X_train,
          Y_train,
          epochs=10,
          batch_size=64,
          validation_data=(X_test, Y_test),
          callbacks=[lr_reducer, early_stopping]
)

Epoch 1/10
1653/1653 [==============================] - 2s 1ms/step - loss: 0.3963 - accuracy: 0.8429 - val_loss: 0.3866 - val_accuracy: 0.8454 - lr: 0.0010
Epoch 2/10
1653/1653 [==============================] - 2s 985us/step - loss: 0.3830 - accuracy: 0.8464 - val_loss: 0.3838 - val_accuracy: 0.8455 - lr: 0.0010
Epoch 3/10
1653/1653 [==============================] - 2s 991us/step - loss: 0.3799 - accuracy: 0.8468 - val_loss: 0.3840 - val_accuracy: 0.8454 - lr: 0.0010
Epoch 4/10
1653/1653 [==============================] - 2s 1ms/step - loss: 0.3781 - accuracy: 0.8469 - val_loss: 0.3809 - val_accuracy: 0.8453 - lr: 0.0010
Epoch 5/10
1653/1653 [==============================] - 2s 1ms/step - loss: 0.3774 - accuracy: 0.8469 - val_loss: 0.3817 - val_accuracy: 0.8450 - lr: 0.0010
Epoch 6/10
1653/1653 [==============================] - 2s 1ms/step - loss: 0.3764 - accuracy: 0.8471 - val_loss: 0.3815 - val_accuracy: 0.8451 - lr: 0.0010
Epoch 7/10
1653/1653 [==============================] 

In [163]:
# compute f1 score
from sklearn.metrics import f1_score

y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print('f1 score: ', f1_score(Y_test, y_pred))

827/827 [==============================] - 0s 427us/step
f1 score:  0.8544660917704674


In [164]:
# save the model
model.save('model.h5')

### Application sur le jeu de test

preprocessing des données de test comme avec train precedemment

In [165]:
# create a x_test and y_test dataframe using the test dataset
# merge test and vendors
test['key'] = 0
vendors['key'] = 0
test = pd.merge(test, vendors, on='key')

# drop the key column
test.drop(['key'], axis=1, inplace=True)

test['CID X LOC X VENDOR'] = test['CID X LOC'] + ' X ' + test['VENDOR'].astype(str)
test.drop(['CID X LOC', 'VENDOR'], axis=1, inplace=True)

print('train shape: ', test.shape)
test.head()

train shape:  (1633300, 88)


,latitude_x,longitude_x,location_type,status,gender,language_x,latitude_y,longitude_y,vendor_category_id,delivery_charge,...,Sweets,Shawarma,Bagels,Sushi,Italian,Vegetarian,Rice,Chinese,Cakes,CID X LOC X VENDOR
0,-96.4,-67.2,1,1,3,0,-0.6,0.8,2.0,0.0,...,0,1,0,0,0,0,0,0,0,ICE2DJP X 0 X 4
1,-96.4,-67.2,1,1,3,0,-0.5,0.7,2.0,0.7,...,0,0,0,0,1,0,0,0,1,ICE2DJP X 0 X 13
2,-96.4,-67.2,1,1,3,0,-0.4,0.6,2.0,0.0,...,0,0,0,0,0,0,0,0,0,ICE2DJP X 0 X 20
3,-96.4,-67.2,1,1,3,0,-0.6,0.8,2.0,0.0,...,0,0,0,0,0,0,0,0,0,ICE2DJP X 0 X 23
4,-96.4,-67.2,1,1,3,0,0.5,0.6,2.0,0.7,...,0,0,0,0,0,0,0,0,0,ICE2DJP X 0 X 28


In [166]:
for col in test.columns:
    print(col, test[col].isnull().sum())

# clean the train dataframe

# add a column "same_language" then drop the language_x and language_y column
test['same_language'] = test['language_x'] == test['language_y']
test.drop(['language_x', 'language_y'], axis=1, inplace=True)

# add a column "is_in_delivery_zone" then drop the latitude_x, longitude_x, latitude_y, longitude_y and serving_distance column
test['is_in_delivery_zone'] = test.apply(lambda x: haversine(x['latitude_x'], x['longitude_x'], x['latitude_y'], x['longitude_y']) <= x['serving_distance'], axis=1)
test.drop(['latitude_x', 'longitude_x', 'latitude_y', 'longitude_y', 'serving_distance'], axis=1, inplace=True)

# convert same_language and is_in_delivery_zone to int
test['same_language'] = test['same_language'].astype(int)
test['is_in_delivery_zone'] = test['is_in_delivery_zone'].astype(int)

latitude_x 0
longitude_x 0
location_type 0
status 0
gender 0
language_x 0
latitude_y 0
longitude_y 0
vendor_category_id 0
delivery_charge 0
serving_distance 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
language_y 0
vendor_rating 0
device_type 0
vendor_orders 0
vendor_customers 0
Salads 0
Thali 0
Japanese 0
Biryani 0
Coffee 0
Indian 0
Pizza 0
Pastas 0
Hot Dogs 0
Rolls 0
Kushari 0
Pizzas 0
Dimsum 0
Frozen yoghurt 0
Seafood 0
Kebabs 0
Shuwa 0
Fatayers 0
Crepes 0
Pancakes 0
Pastry 0
Waffles 0
Combos 0
Family Meal 0
Organic 0
Hot Chocolate 0
Soups 0
Pasta 0
American 0
Mandazi 0
Manakeesh 0
Karak 0
Free Delivery 0
Sandwiches 0
Asian 0
Fresh Juices 0
Milkshakes 0
Fries 0
Burgers 0
Lebanese 0
Desserts 0
Healthy Food 0
Thai 0
Arabic 0
Churros 0
Grills 0
Smoothies 0
Cafe 0
Kids meal 0
Steaks 0
Mexican 0
Omani 0
Breakfast 0
Mojitos 0
Ice creams 0
Donuts 0
Mishkak 0
Spanish Latte 0
Sweets 0
Shawarma 0
Bagels 0
Sushi 0
Italian 0
Vegetarian 0
Rice 0
Chinese 0
Cakes 0
CID X LOC X VENDOR

In [167]:
for col in test.columns:
    print(col, test[col].isnull().sum())

# keep the CID X LOC X VENDOR column in a vector
CID_X_LOC_X_VENDOR = test['CID X LOC X VENDOR']

# drop CID X LOC X VENDOR column
test.drop(['CID X LOC X VENDOR'], axis=1, inplace=True)

location_type 0
status 0
gender 0
vendor_category_id 0
delivery_charge 0
is_open 0
prepration_time 0
discount_percentage 0
rank 0
vendor_rating 0
device_type 0
vendor_orders 0
vendor_customers 0
Salads 0
Thali 0
Japanese 0
Biryani 0
Coffee 0
Indian 0
Pizza 0
Pastas 0
Hot Dogs 0
Rolls 0
Kushari 0
Pizzas 0
Dimsum 0
Frozen yoghurt 0
Seafood 0
Kebabs 0
Shuwa 0
Fatayers 0
Crepes 0
Pancakes 0
Pastry 0
Waffles 0
Combos 0
Family Meal 0
Organic 0
Hot Chocolate 0
Soups 0
Pasta 0
American 0
Mandazi 0
Manakeesh 0
Karak 0
Free Delivery 0
Sandwiches 0
Asian 0
Fresh Juices 0
Milkshakes 0
Fries 0
Burgers 0
Lebanese 0
Desserts 0
Healthy Food 0
Thai 0
Arabic 0
Churros 0
Grills 0
Smoothies 0
Cafe 0
Kids meal 0
Steaks 0
Mexican 0
Omani 0
Breakfast 0
Mojitos 0
Ice creams 0
Donuts 0
Mishkak 0
Spanish Latte 0
Sweets 0
Shawarma 0
Bagels 0
Sushi 0
Italian 0
Vegetarian 0
Rice 0
Chinese 0
Cakes 0
CID X LOC X VENDOR 0
same_language 0
is_in_delivery_zone 0


test du modèle avec le jeu de test et création du fichier Submission.csv

In [168]:
# if needed
model = keras.models.load_model('model.h5')

In [169]:
# predict the test dataset
y_pred = model.predict(test)

  354/51041 [..............................] - ETA: 21s

2023-05-16 15:17:20.042211: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1071444800 exceeds 10% of free system memory.


51041/51041 [==============================] - 22s 422us/step


In [170]:
# flatten the y_pred
y_pred = y_pred.flatten()

# convert the y_pred to int (0 or 1)
y_pred = y_pred.round().astype(int)

In [171]:
# create the submission dataframe
submission = pd.DataFrame({'CID X LOC X VENDOR': CID_X_LOC_X_VENDOR, 'target': y_pred.flatten()})

# save the submission dataframe
submission.to_csv('Submission.csv', index=False)